In [26]:
#!pip3 install tensorflow_model_optimization

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Flatten, Dropout, Dense
import tensorflow_model_optimization as tfmot

In [5]:
(x_train,y_train),(x_test,y_test)=keras.datasets.mnist.load_data()

In [11]:
x_train=x_train/255
x_test=x_test/255

In [13]:
model=tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation='sigmoid'),
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 1.1944 - accuracy: 0.7179
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4933 - accuracy: 0.8726
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3862 - accuracy: 0.8942
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3429 - accuracy: 0.9035
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3179 - accuracy: 0.9094


In [22]:
model.save('./mnist_model')

INFO:tensorflow:Assets written to: ./mnist_model/assets


INFO:tensorflow:Assets written to: ./mnist_model/assets


# Weight Quantization + tf.Lite

In [32]:
converter=tf.lite.TFLiteConverter.from_saved_model("./mnist_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT] 
tflite_quant_model = converter.convert()

with open('tflite_quant_model.tflite', "wb") as f:
    f.write(tflite_quant_model)

2023-11-17 14:53:58.799026: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-17 14:53:58.799044: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-17 14:53:58.799240: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./mnist_model
2023-11-17 14:53:58.799997: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-17 14:53:58.800007: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: ./mnist_model
2023-11-17 14:53:58.802112: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-17 14:53:58.832828: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: ./mnist_model
2023-11-17 14:53:58.841897: I tensorflow/cc/saved_model/loader.cc:316] SavedModel load for tags { serve }; Status: success: OK. Took 42657 microsecon

# Model Quantization + Training + tf.lite

In [37]:
quantize_model=tfmot.quantization.keras.quantize_model
q_model=quantize_model(model)
q_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

q_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 28, 28)            3         
 yer)                                                            
                                                                 
 quant_flatten_1 (QuantizeW  (None, 784)               1         
 rapperV2)                                                       
                                                                 
 quant_dense (QuantizeWrapp  (None, 100)               78505     
 erV2)                                                           
                                                                 
 quant_dense_1 (QuantizeWra  (None, 10)                1015      
 pperV2)                                                         
                                                                 
Total params: 79524 (310.64 KB)
Trainable params: 79510 

In [38]:
q_model.fit(x_train, y_train, epochs=2)

Epoch 1/2
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3019 - accuracy: 0.9133
Epoch 2/2
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2865 - accuracy: 0.9173


In [39]:
converter=tf.lite.TFLiteConverter.from_keras_model(q_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT] 
tflite_q_aware_model = converter.convert()

with open('tflite_q_aware_model.tflite', "wb") as f:
    f.write(tflite_q_aware_model)

INFO:tensorflow:Assets written to: /var/folders/6p/5mzp7fyd4mgchv364j3jy2jr0000gn/T/tmpemjr1n9b/assets


INFO:tensorflow:Assets written to: /var/folders/6p/5mzp7fyd4mgchv364j3jy2jr0000gn/T/tmpemjr1n9b/assets
/usr/local/lib/python3.11/site-packages/tensorflow/lite/python/convert.py:947: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2023-11-17 15:04:40.566592: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-17 15:04:40.566614: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-17 15:04:40.566872: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/6p/5mzp7fyd4mgchv364j3jy2jr0000gn/T/tmpemjr1n9b
2023-11-17 15:04:40.569241: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-17 15:04:40.569253: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/6p/5mzp7fyd4mgchv364j3jy2jr0000gn/T/tmpemjr1n9b